In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import random
from time import sleep
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

from soporte import crear_df

In [2]:
lista_municipios = ['acebeda-la', 'ajalvir', 'alameda-del-valle', 'alamo-el', 'alcala-de-henares', 'alcobendas', 'alcorcon', 'aldea-del-fresno', 'algete', 'alpedrete', 'ambite', 'anchuelo', 'aranjuez', 'arganda-del-rey', 'arroyomolinos', 'atazar-el', 'batres', 'becerril-de-la-sierra', 'belmonte-de-tajo', 'berrueco-el', 'berzosa-del-lozoya', 'boadilla-del-monte', 'boalo-el', 'braojos', 'brea-de-tajo', 'brunete', 'buitrago-del-lozoya', 'bustarviejo', 'cabanillas-de-la-sierra', 'cabrera-la', 'cadalso-de-los-vidrios', 'camarma-de-esteruelas', 'campo-real', 'canencia', 'carabana', 'casarrubuelos', 'cenicientos', 'cercedilla', 'cervera-de-buitrago', 'chapineria', 'chinchon', 'ciempozuelos', 'cobena', 'collado-mediano', 'collado-villalba', 'colmenar-del-arroyo', 'colmenar-de-oreja', 'colmenarejo', 'colmenar-viejo', 'corpa', 'coslada', 'cubas-de-la-sagra', 'daganzo-de-arriba', 'escorial-el', 'estremera', 'fresnedillas-de-la-oliva', 'fresno-de-torote', 'fuenlabrada', 'fuente-el-saz-de-jarama', 'fuentiduena-de-tajo', 'galapagar', 'garganta-de-los-montes', 'gargantilla-del-lozoya-y-pinilla-de-buitrago', 'gascones', 'getafe', 'grinon', 'guadalix-de-la-sierra', 'guadarrama', 'hiruela-la', 'horcajo-de-la-sierra-aoslos', 'horcajuelo-de-la-sierra', 'hoyo-de-manzanares', 'humanes-de-madrid', 'leganes', 'loeches', 'lozoya', 'lozoyuela-navas-sieteiglesias', 'madarcos', 'madrid', 'majadahonda', 'manzanares-el-real', 'meco', 'mejorada-del-campo', 'miraflores-de-la-sierra', 'molar-el', 'molinos-los', 'montejo-de-la-sierra', 'moraleja-de-enmedio', 'moralzarzal', 'morata-de-tajuna', 'mostoles', 'navacerrada', 'navalafuente', 'navalagamella', 'navalcarnero', 'navarredonda-y-san-mames', 'navas-del-rey', 'nuevo-baztan', 'olmeda-de-las-fuentes', 'orusco-de-tajuna', 'paracuellos-de-jarama', 'parla', 'patones', 'pedrezuela', 'pelayos-de-la-presa', 'perales-de-tajuna', 'pezuela-de-las-torres', 'pinilla-del-valle', 'pinto', 'pinuecar-gandullas', 'pozuelo-de-alarcon', 'pozuelo-del-rey', 'pradena-del-rincon', 'puebla-de-la-sierra', 'puentes-viejas-manjiron', 'quijorna', 'rascafria', 'reduena', 'ribatejada', 'rivas-vaciamadrid', 'robledillo-de-la-jara', 'robledo-de-chavela', 'robregordo', 'rozas-de-madrid-las', 'rozas-de-puerto-real', 'san-agustin-del-guadalix', 'san-fernando-de-henares', 'san-lorenzo-de-el-escorial', 'san-martin-de-la-vega', 'san-martin-de-valdeiglesias', 'san-sebastian-de-los-reyes', 'santa-maria-de-la-alameda', 'santorcaz', 'santos-de-la-humosa-los', 'serna-del-monte-la', 'serranillos-del-valle', 'sevilla-la-nueva', 'somosierra', 'soto-del-real', 'talamanca-de-jarama', 'tielmes', 'titulcia', 'torrejon-de-ardoz', 'torrejon-de-la-calzada', 'torrejon-de-velasco', 'torrelaguna', 'torrelodones', 'torremocha-de-jarama', 'torres-de-la-alameda', 'tres-cantos', 'valdaracete', 'valdeavero', 'valdelaguna', 'valdemanco', 'valdemaqueda', 'valdemorillo', 'valdemoro', 'valdeolmos-alalpardo', 'valdepielagos', 'valdetorres-de-jarama', 'valdilecha', 'valverde-de-alcala', 'velilla-de-san-antonio', 'vellon-el', 'venturada', 'villaconejos', 'villa-del-prado', 'villalbilla', 'villamanrique-de-tajo', 'villamanta', 'villamantilla', 'villanueva-de-la-canada', 'villanueva-del-pardillo', 'villanueva-de-perales', 'villar-del-olmo', 'villarejo-de-salvanes', 'villaviciosa-de-odon', 'villavieja-del-lozoya', 'zarzalejo']

In [3]:
municipios_aleatorios = random.choices(lista_municipios, k = 5)
municipios_aleatorios

['rozas-de-puerto-real',
 'hiruela-la',
 'brea-de-tajo',
 'molinos-los',
 'fuente-el-saz-de-jarama']

In [4]:
driver =webdriver.Chrome()
url_bueno= "https://www.wunderground.com/weather/es/collado-villalba"
driver.get(url_bueno)
driver.maximize_window()

# Dar 10 segundos hasta encontrar la ventana de cookies
iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located(('xpath','//*[@id="sp_message_iframe_1165301"]')))
# Cambiar al 'iframe' de cookies
driver.switch_to.frame(iframe)
# Esperar
sleep(3)
# Rechazar cookies
driver.find_element("css selector","#notice > div.message-component.message-row.cta-buttons-container > div.message-component.message-column.cta-button-column.reject-column > button").click()

# Vovler al contenido principal
driver.switch_to.default_content() 

# Esperar
sleep(3)
# Hacer click en la estación
driver.find_element("css selector", "#inner-content > div.region-content-top > lib-city-header > div:nth-child(1) > div > div > a.station-name").click()
# Esperar
sleep(3)
# Tomar el código
codigo_municipio = driver.find_element("css selector", "#inner-content > div.region-content-top > app-dashboard-header > div.dashboard__header.small-12.ng-star-inserted > div > div.heading > h1").text.split(' - ')[1]

dfs = []
tablas = []
"""
driver.find_element("css selector", "#modeSelect").click()
driver.find_element("css selector", "#modeSelect > option:nth-child(3)").click()
driver.find_element("css selector", "#main-page-content > div > div > div > lib-history > div.navigation-wrapper > div.selects > button").click()
driver.find_element("css selector", "#history-tab-group > li:nth-child(2) > a").click()
tabla = driver.find_element("css selector", "#main-page-content > div > div > div > lib-history > div.history-tabs > lib-history-table > div > div").text
tablas.append(tabla)
df = crear_df(tabla)
dfs.append(df)


for i in range(1,3):
    #Redirect
    val = f'https://www.wunderground.com/dashboard/pws/{codigo_municipio}/table/2024-{i}-17/2024-{i}-17/monthly'
    wait = WebDriverWait(driver, 10)
    driver.get(val)
    wait.until(EC.url_to_be(val))
    print(f'estamos en {i}')
    tabla = driver.find_element("css selector", "#main-page-content > div > div > div > lib-history > div.history-tabs > lib-history-table > div > div").text
    tablas.append(tabla)
    df = crear_df(tabla)
    dfs.append(df)"""

#driver.close()


'\ndriver.find_element("css selector", "#modeSelect").click()\ndriver.find_element("css selector", "#modeSelect > option:nth-child(3)").click()\ndriver.find_element("css selector", "#main-page-content > div > div > div > lib-history > div.navigation-wrapper > div.selects > button").click()\ndriver.find_element("css selector", "#history-tab-group > li:nth-child(2) > a").click()\ntabla = driver.find_element("css selector", "#main-page-content > div > div > div > lib-history > div.history-tabs > lib-history-table > div > div").text\ntablas.append(tabla)\ndf = crear_df(tabla)\ndfs.append(df)\n\n\nfor i in range(1,3):\n    #Redirect\n    val = f\'https://www.wunderground.com/dashboard/pws/{codigo_municipio}/table/2024-{i}-17/2024-{i}-17/monthly\'\n    wait = WebDriverWait(driver, 10)\n    driver.get(val)\n    wait.until(EC.url_to_be(val))\n    print(f\'estamos en {i}\')\n    tabla = driver.find_element("css selector", "#main-page-content > div > div > div > lib-history > div.history-tabs > 

In [6]:
for i in range(1,3):
    #Redirect
    val = f'https://www.wunderground.com/dashboard/pws/ICOLLA28/table/2024-{i}-17/2024-{i}-17/monthly'
    wait = WebDriverWait(driver, 10)
    driver.get(val)
    wait.until(EC.url_to_be(val))
    print(f'estamos en {i}')
    tabla = driver.find_element("css selector", "#main-page-content > div > div > div > lib-history > div.history-tabs > lib-history-table > div > div").text
    tablas.append(tabla)
    print('he')
    df = crear_df(tabla)
    print('lolo')
    dfs.append(df)

estamos en 1
he
lolo


/Users/javi/Documents/Hackio/Laboratorios/Laboratorio-Selenium/soporte.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: float(x))


estamos en 2
he
lolo


/Users/javi/Documents/Hackio/Laboratorios/Laboratorio-Selenium/soporte.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: float(x))


In [11]:
dfs[0]

,High Temp (ºF),Avg Temp (ºF),Low T (ºF),High Dew Pt (ºF),Avg Dew Pt (ºF),Low Dew Pt (ºF),High Hum (%),Avg Hum (%),Low Hum (%),High Spe (mph),Avg Spe (mph),Low Spe (mph),High Press (in),Low Press (in),Sum Prec (In)
Date,,,,,,,,,,,,,,,
1/1/2024,48.7,40.6,33.8,38.8,33.7,29.1,92.0,77.0,50.0,10.2,1.4,0.0,29.93,29.81,0.00
1/2/2024,46.5,38.6,33.4,45.8,36.7,32.3,98.0,93.0,86.0,6.9,0.9,0.0,29.93,29.85,0.00
1/3/2024,55.2,48.6,44.7,50.1,47.0,41.7,99.0,95.0,80.0,10.7,1.6,0.0,29.87,29.81,0.19
1/4/2024,47.4,44.2,40.2,46.2,42.7,39.5,99.0,95.0,83.0,9.6,1.2,0.0,29.87,29.58,0.11
1/5/2024,44.4,39.5,37.0,41.0,34.7,30.0,95.0,83.0,59.0,22.5,5.7,0.0,29.64,29.50,0.09
1/6/2024,42.9,38.2,33.6,33.9,31.2,28.4,88.0,76.0,62.0,22.3,4.3,0.0,29.88,29.64,0.00
1/7/2024,46.4,38.1,32.5,31.2,28.3,25.1,83.0,69.0,44.0,11.8,3.0,0.0,29.92,29.86,0.00
1/8/2024,44.9,35.4,26.2,32.9,28.6,24.0,92.0,77.0,53.0,6.7,0.9,0.0,29.90,29.80,0.00
1/9/2024,41.3,38.1,35.0,37.0,34.5,31.8,94.0,87.0,79.0,8.9,1.3,0.0,29.83,29.78,0.00
